In [4]:
pip install bs4


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install html5lib


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from bs4 import BeautifulSoup
import time
import requests
import re

In [9]:

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
#driver = webdriver.Chrome()
#driver.get(URL)
#time.sleep(3)
#html = driver.page_source
#soup = BeautifulSoup(html, 'html.parser')
#r = requests.get(url=URL, headers=headers)
#driver.switch_to.frame("sp_message_iframe_764224") 
#element = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[3]/div[2]/button').click()

In [ ]:
element = driver.find_element(by=By.XPATH, value= '//*[title="ACCEPT ALL]' )
element.click()

In [ ]:
players_list = []
age_list = []
position_list = []
value_list = []
badge_list = []

In [ ]:
for pagenum in range(1, 5):
    URL = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/page/" +str(pagenum)
    r = requests.get(url=URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    players = soup.find_all("td", class_="hauptlink")
    ages = soup.find_all("td", class_="zentriert")
    nationality = soup.find_all("td", class_="zentriert")
    values = soup.find_all("td", class_="rechts hauptlink")
    

In [ ]:
for pagenum in range(1, 5):
    URL = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/page/" + str(pagenum)
    r = requests.get(url=URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    players = soup.find_all("td", class_="hauptlink")
    for player in players:
        players_list.append(player.text)

    ages = soup.find_all("td", class_="zentriert")
    for age in ages:
        age_list.append(age.text)
        
    nationalities = soup.find_all("td", class_="zentriert")
    for nation in nationality:
        flags = nation.find_all('img')  
        if flags:  
            first_flag = flags[0]  
            first_nationality = first_flag['title']
            nationality_list.append(first_nationality)
    
    values = soup.find_all("td", class_="rechts hauptlink")
    for value in values:
        value_list.append(value.text)

In [ ]:
badges = soup.find_all("td", class_="zentriert")
for badge in badges:
    flags = badge.find_all('img')  
    if flags:  
        first_flag = flags[0]  
        first_badge = first_flag['title']
        badge_list.append(first_badge)
        print(first_nationality)

In [ ]:
nationality_list

In [ ]:
players

In [ ]:
for player in players:
    players_list.append(player.text)

In [ ]:
players_list

In [ ]:

i = 1
while i < len(players_list):
    del players_list[i]
    i += 1
print(players_list)


In [ ]:
ages = soup.find_all("td", class_="zentriert")

In [ ]:
ages

In [ ]:
for age in ages:
    age_list.append(age.text)

In [ ]:
age_list

In [ ]:
new_age_list = []
for i in range(2, len(age_list), 4):
    new_age_list.append(age_list[i])
print(new_age_list)

In [ ]:
new_age_list

In [ ]:
nationality = soup.find_all("td", class_="zentriert")

In [ ]:
nationality

In [ ]:
for pagenum in range(1, 5):
    URL = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/page/" + str(pagenum)
    r = requests.get(url=URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table", class_='items')
    tbody = table.find("tbody")
    badges = tbody.find_all("td", class_="zentriert")
    for badge in badges:
        flags = badge.find_all('img')  
        if flags:  
            first_flag = flags[0]  
            first_badge = first_flag['title']
            badge_list.append(first_badge)
            print(first_badge)


In [ ]:
badge_list

In [ ]:
del badge_list[-5:]

In [ ]:
indexes_to_delete = [219, 218, 217, 216, 215, 164, 163, 162, 161, 160, 109, 108, 107, 106, 105, 54, 53, 52, 51, 50]

for index in indexes_to_delete:
    del badge_list[index]

In [ ]:
badge_list

In [ ]:
nationality_list = badge_list[::2]
club_list = badge_list[1::2]

In [ ]:
nationality_list

In [ ]:
club_list

In [ ]:
for nation in nationality:
    flags = nation.find('img')  
    if flags:  
        first_flag = flags['0']  
        first_nationality = first_flag['title']
        nationality_list.append(first_nationality)
        print(first_nationality)

In [ ]:
i = 1
while i < len(nationality_list):
    del nationality_list[i]
    i += 1
print(nationality_list)

In [ ]:
values = soup.find_all("td", class_="rechts hauptlink")

In [ ]:
values

In [ ]:
for value in values:
    value_list.append(value.text)

In [ ]:
value_list

In [ ]:
all_players = []

for pagenum in range(1, 5):
    URL = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/page/" + str(pagenum)
    r = requests.get(url=URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table", class_='items')
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")

    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        all_players.append(cols)

    print(all_players)
    print(len(all_players))


In [ ]:
all_players

In [ ]:
final_players = [all_players[i] for i in range(len(all_players)) if i % 3 == 0]

print(final_players)


In [ ]:
final_players[2]

In [ ]:
import pandas as pd

df = pd.DataFrame(final_players, columns=['Rank', 'null', 'null1', 'Player', 'Position', 'null3', 'Age', 'null4', 'Value'])


In [ ]:
df.head()

In [ ]:
df['Nationality'] = nationality_list

In [ ]:
df['Club'] = club_list

In [ ]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
URL_TEMPLATE = 'https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/page/{}'
headers = {'User-Agent': USER_AGENT}

In [ ]:
def scrape_webpage(url):
    response = requests.get(url=url, headers=headers)
    return BeautifulSoup(response.content, 'html.parser')

In [ ]:
all_players = []
for pagenum in range(1, 5):
    url = URL_TEMPLATE.format(pagenum)
    soup = scrape_webpage(url)
    table = soup.find("table", class_='items')
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        all_players.append(cols)
final_players = [all_players[i] for i in range(len(all_players)) if i % 3 == 0]
print(final_players)

In [ ]:
all_players = []
for pagenum in range(1, 5):
    URL = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/page/" + str(pagenum)
    r = requests.get(url=URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table", class_='items')
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")

    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        all_players.append(cols)

final_players = [all_players[i] for i in range(len(all_players)) if i % 3 == 0]

In [1]:
league_codes = ["AN1L", "AR1N", "AUS1", "A1", "BGD1", "BE1", "BO1A", "BRA1", "CDN1", "CLPD", "CSL", "COLP", "COL1", "CRPD", "PDV1", "KR1", "TS1",
"DK1", 
"EL1A", 
"EL1S", 
"SL1A", 
"GB1", 
"GB2", 
"GB3", 
"GB4", 
"FIJ1", 
"FR1", 
"FR2", 
"L1", 
"L2", 
"L3", 
"GHPL", 
"GR1", 
"GU1A", 
"GU1C", 
"HGKG", 
"UNG1", 
"IND1", 
"IN1L", 
"IRN1", 
"IR1", 
"ISR1", 
"IT1", 
"IT2", 
"JPL1", 
"JAP1", 
"RSK1", 
"KG1L", 
"MYS1", 
"MEXA", 
"MARP", 
"MO1L", 
"MYA1", 
"NPL1", 
"NL1", 
"NCL1", 
"NNL1", 
"NO1", 
"PN1A", 
"PN1C", 
"PR1A", 
"PR1C", 
"TDeA", 
"TDeC", 
"PFL1", 
"PL1", 
"PO1", 
"QSL", 
"RO1", 
"RU1", 
"SA1", 
"SC1", 
"SER1", 
"SIN1", 
"SFA1", 
"ES1", 
"ES2", 
"SE1", 
"C1", 
"TAD1", 
"THA1", 
"TUN1", 
"TUSC", 
"TR1", 
"UKR1", 
"UAE1", 
"MLS1", 
"URU1", 
"UZ1", 
"VZ1L", 
"VIE1"]
list_seasons = ["2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]

In [2]:
url_temp = "https://www.transfermarkt.co.uk/jumplist/startseite/wettbewerb/{}"


In [10]:
list_urls = []
for codes in league_codes:
    url = url_temp.format(codes)
    print(url)
    r = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table", class_='items')
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    for row in rows:
        team_links = row.select('a')[0]['href']
        print(team_links)
        list_urls.append("https://www.transfermarkt.co.uk" + team_links)

https://www.transfermarkt.co.uk/jumplist/startseite/wettbewerb/AN1L
/ca-petroleos-luanda/startseite/verein/7084/saison_id/2020
/cd-1-ordm-de-agosto/startseite/verein/14503/saison_id/2020
/fc-onze-bravos-do-maquis/startseite/verein/35599/saison_id/2020
/gd-interclube-luanda/startseite/verein/9539/saison_id/2020
/grupo-desportivo-sagrada-esperanca/startseite/verein/26741/saison_id/2020
/desportivo-da-huila/startseite/verein/39419/saison_id/2020
/recreativo-do-libolo/startseite/verein/26740/saison_id/2020
/academica-petroleos-do-lobito/startseite/verein/46565/saison_id/2020
/progresso-do-sambizanga/startseite/verein/35308/saison_id/2020
/sporting-clube-petroleos-de-cabinda/startseite/verein/36832/saison_id/2020
/clube-recreativo-da-caala/startseite/verein/24712/saison_id/2020
/santa-rita-de-cassia-fc/startseite/verein/60370/saison_id/2020
/cuando-cubango-fc/startseite/verein/87070/saison_id/2020
/gd-herois-da-baixa-de-kassanje/startseite/verein/88759/saison_id/2020
/wiliete-sport-clube-de

In [11]:
len(list_urls)

1380

In [17]:
list_player_url = set()
for link in list_urls:
    print(link)
    r = requests.get(url=link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table", class_='items')

    if table is not None:
        tbody = table.find("tbody")
        rows = tbody.find_all("tr")
        for row in rows:
            cells = row.find_all('td', 'hauptlink')
            if cells:
                player_link = cells[0].find('a')['href']
                list_player_url.add("https://www.transfermarkt.co.uk" + player_link)
    else:
        print("Table with class 'items' not found on the page.")






    """rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td', 'hauptlink')
        if cells:
            player_link = cells[0].find('a')['href']
            list_player_url.add("https://www.transfermarkt.co.uk" + player_link)"""

list_player_url = list(list_player_url)


https://www.transfermarkt.co.uk/ca-petroleos-luanda/startseite/verein/7084/saison_id/2020
https://www.transfermarkt.co.uk/cd-1-ordm-de-agosto/startseite/verein/14503/saison_id/2020
https://www.transfermarkt.co.uk/fc-onze-bravos-do-maquis/startseite/verein/35599/saison_id/2020
https://www.transfermarkt.co.uk/gd-interclube-luanda/startseite/verein/9539/saison_id/2020
https://www.transfermarkt.co.uk/grupo-desportivo-sagrada-esperanca/startseite/verein/26741/saison_id/2020
https://www.transfermarkt.co.uk/desportivo-da-huila/startseite/verein/39419/saison_id/2020
https://www.transfermarkt.co.uk/recreativo-do-libolo/startseite/verein/26740/saison_id/2020
https://www.transfermarkt.co.uk/academica-petroleos-do-lobito/startseite/verein/46565/saison_id/2020
https://www.transfermarkt.co.uk/progresso-do-sambizanga/startseite/verein/35308/saison_id/2020
https://www.transfermarkt.co.uk/sporting-clube-petroleos-de-cabinda/startseite/verein/36832/saison_id/2020
https://www.transfermarkt.co.uk/clube-re

In [20]:
list_player_url[1]

'https://www.transfermarkt.co.uk/manuel-corozo/profil/spieler/719630'